In [2]:
import pandas as pd

print(pd.__version__)


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('future.no_silent_downcasting', True)

2.2.2


In [3]:
from datetime import datetime, timedelta

def generate_month_year_list(start_year, start_month, n_months=36):
    start_month = datetime(year=start_year, month=start_month, day=1)
    month_year_list = []

    for i in range(n_months):
        # Format the month and year as "Month Year"
        month_year = start_month.strftime("%B %Y")
        month_year_list.append(month_year)
        
        # Move to the previous month
        # Use timedelta to avoid issues with months of varying lengths
        first_day_of_current_month = start_month.replace(day=1)
        start_month = first_day_of_current_month - timedelta(days=1)
    
    return month_year_list

# generate_month_year_list(2024, 8)

In [4]:
import requests

# URL of the file to be downloaded
url = 'https://am.jpmorgan.com/content/dam/jpm-am-aem/emea/gb/en/supplemental/full-portfolio-listing/jpm-emerging-europe-middle-east-afria-disclosure.xlsx'

# Specify the local filename to save the downloaded file
local_filename = 'jpm-emerging-europe-middle-east-afria-disclosure.xlsx'

# Send a HTTP GET request to the URL
with requests.get(url, stream=True) as response:
    response.raise_for_status()  # Check for HTTP errors

    # Open a local file with write-binary mode
    with open(local_filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

print(f'File downloaded: {local_filename}')

File downloaded: jpm-emerging-europe-middle-east-afria-disclosure.xlsx


In [11]:
current_month_idx = 9 # Sep
year = 2024

month_years = generate_month_year_list(year, current_month_idx)
# month_years = ['March 2022', 'February 2022']
# month_years = ['February 2023', 'January 2023'] #, 'December 2022']
# print(month_years)

# set up structure of df
description_col = 'Security Description'
security_col = 'Security No.'
holding_col = 'Holding'
value_col = 'Market Value'
percent_col = '% of Fund'
sheet_name = month_years[0]

jema_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name=sheet_name, skiprows=9, usecols=range(5))
jema_data = jema_data.dropna()
jema_data = jema_data.drop(columns=[holding_col, value_col, percent_col])

# get all unique entries by security_col
for month_year in month_years:
    sheet_name = month_year
    try:
        month_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name=sheet_name, skiprows=9, usecols=range(5))
        month_data = month_data.drop(columns=[holding_col, value_col, percent_col]).dropna()
        month_data[security_col] = month_data[security_col].replace('0H6400S', '2H6442S')

        # fix LUKOIL issue - drop the entries with the wrong security code, they'll be amended below
        month_data = month_data[month_data[security_col] != '0H6400S']
        
        # fix Rosneft issue - drop the entries with the wrong security code, they'll be amended below
        month_data = month_data[month_data[security_col] != '0H6367S']

        # drop subtotal row
        month_data = month_data[month_data[description_col] != 'Subtotal']

        jema_data = pd.concat([jema_data, month_data], ignore_index=True)
    except Exception as e:
        print(f"{sheet_name} failed")

print()

jema_symbols_exchanges = pd.read_csv('./jema_symbols_exchanges.csv').dropna(subset=['Symbol'])
jema_symbols_exchanges.reset_index(inplace=True, drop=True)
jema_symbols_exchanges = jema_symbols_exchanges.drop(columns=[description_col])
jema_symbols_exchanges = jema_symbols_exchanges.dropna()

jema_data = pd.merge(jema_data, jema_symbols_exchanges, on=security_col, how='left')

jema_data = jema_data.drop_duplicates([security_col])
jema_data = jema_data.reset_index(drop=True)
jema_data['Conv Rate'] = jema_data['Conv Rate'].fillna(1)
jema_data = jema_data.fillna('')

num_rows = len(jema_data)

jema_data_old = jema_data

def read_jema_sheet(jema_data, sheet_name, security_col):
        month_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name=sheet_name, skiprows=9, usecols=range(5)).dropna()
        month_data = month_data.dropna()
        month_holding_col = f'{sheet_name}'
        month_value_col = f'{value_col} {sheet_name}'
        month_percent_col = f'{percent_col} {sheet_name}'

        month_data.rename(columns={holding_col: month_holding_col}, inplace=True)
        month_data.rename(columns={value_col:   month_value_col}, inplace=True)
        month_data.rename(columns={percent_col: month_percent_col}, inplace=True)
        month_data = month_data.dropna()

        # fix LUKOIL issue - amend wrong security code
        month_data[security_col] = month_data[security_col].replace('0H6400S', '2H6442S')

        # fix Rosneft issue  - amend wrong security code
        month_data[security_col] = month_data[security_col].replace('0H6367S', '2H7674S')

        # fix Novatek issue - amend wrong security code
        month_data.loc[(month_data[security_col] == '2H6464S') & (month_data[month_holding_col] ==  103572), security_col] = '0H5828S'

        # drop subtotal row
        month_data = month_data[month_data[description_col] != 'Subtotal']

        jema_data = jema_data.merge(month_data[[month_holding_col, security_col]], on=security_col, how='left')
        jema_data.fillna(0, inplace=True)

        return jema_data


for month_year in month_years:
    sheet_name = month_year
    try:
        jema_data = read_jema_sheet(jema_data, sheet_name, security_col)
        if(len(jema_data_old) != len(jema_data)):
            duplicates = jema_data[jema_data[security_col].duplicated(keep=False)]

            print(f"\nRows with duplicate names {sheet_name}:")
            print(duplicates)

        jema_data_old = jema_data

    except Exception as e:
        print(f"{sheet_name} failed")

jema_data.to_csv("jema_holdings_full.csv")
jema_data


February 2022 failed
January 2022 failed
December 2021 failed
November 2021 failed
October 2021 failed

February 2022 failed
January 2022 failed
December 2021 failed
November 2021 failed
October 2021 failed


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,September 2024,August 2024,July 2024,June 2024,May 2024,April 2024,March 2024,February 2024,January 2024,December 2023,November 2023,October 2023,September 2023,August 2023,July 2023,June 2023,May 2023,April 2023,March 2023,February 2023,January 2023,December 2022,November 2022,October 2022,September 2022,August 2022,July 2022,June 2022,May 2022,April 2022,March 2022
0,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1.0,32585,32585,32585,32585,32585,43283,43283,43283,50214,48261,52402,52402,53763,53763,53763,53763,53763,53763,47835,47835,0,0,0,0,0,0,0,0,0,0,0
1,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1.0,514330.73,514330.73,514330.73,514330.73,514330.73,514330.73,514330.73,514330.73,500786.64,1000786.64,1000786.64,1000786.64,1000786.64,1000786.64,785612.19,1000057.54,1000057.54,1000057.54,1005957.54,2029792.33,17002248.27,17002248.27,17002248.27,16981316.3,16981316.3,17068108.68,17073471.32,0,0,0,0
2,QATAR NATIONAL BANK QPSC COMMON STOCK QAR 1,6148197,QNBK,QSE,QAR,1.0,142756,142756,142756,117376,117376,117376,117376,131277,131277,131277,131277,131277,131277,145911,145911,145911,145911,145911,145911,145911,0,0,0,0,0,0,0,0,0,0,0
3,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAC,1.0,121145,164763,164763,164763,164763,164763,164763,164763,164763,164763,164763,164763,193157,193157,193157,193157,165940,165940,159093,159093,0,0,0,0,0,0,0,0,0,0,0
4,SAUDI ARABIAN OIL CO COMMON STOCK SAR,BJTM270,2222,TADAWUL,SAR,1.0,73566,73566,73566,73566,73566,73566,73566,95550,95550,95550,95550,95550,84432,84432,84432,84432,84432,76757,76757,76757,0,0,0,0,0,0,0,0,0,0,0
5,NASPERS LTD COMMON STOCK ZAR 2,BN0VX82,NPN,JSE,ZAC,1.0,2142,2142,2142,2142,2142,0,0,2744,2744,2744,4185,4185,4185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAC,1.0,35497,47354,47354,47354,47354,47354,47354,47354,47354,47354,47354,47354,54674,45012,45012,45012,45012,45012,45012,45012,0,0,0,0,0,0,0,0,0,0,0
7,EMAAR PROPERTIES PJSC COMMON STOCK AED 1,B01RM25,EMAAR,DFM,AED,1.0,210406,225483,225483,200673,200673,200673,200673,200673,200673,200673,200673,244040,244040,244040,244040,244040,244040,244040,244040,244040,0,0,0,0,0,0,0,0,0,0,0
8,GOLD FIELDS LTD COMMON STOCK ZAR 50,6280215,0QQ8,LSE,CHF,1.0,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,31177,18714,18714,0,0,0,0,0,0,0,0,0,0,0
9,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1.0,51456,51456,51456,81271,81271,81271,81271,81271,81271,81271,81271,81271,84089,90825,90825,90825,90825,67786,67786,37122,0,0,0,0,0,0,0,0,0,0,0


In [16]:
latest_data = jema_data.iloc[:, :7]
latest_data = latest_data.rename(columns={'September 2024': 'Holding'})
latest_data = latest_data[latest_data['Holding'] != 0]
display(latest_data)
latest_data.to_csv("../nav_ninja/data/jema.csv")

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding
0,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1.0,32585
1,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1.0,514330.73
2,QATAR NATIONAL BANK QPSC COMMON STOCK QAR 1,6148197,QNBK,QSE,QAR,1.0,142756
3,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAC,1.0,121145
4,SAUDI ARABIAN OIL CO COMMON STOCK SAR,BJTM270,2222,TADAWUL,SAR,1.0,73566
5,NASPERS LTD COMMON STOCK ZAR 2,BN0VX82,NPN,JSE,ZAC,1.0,2142
6,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAC,1.0,35497
7,EMAAR PROPERTIES PJSC COMMON STOCK AED 1,B01RM25,EMAAR,DFM,AED,1.0,210406
8,GOLD FIELDS LTD COMMON STOCK ZAR 50,6280215,0QQ8,LSE,CHF,1.0,31177
9,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1.0,51456


OSError: Cannot save file into a non-existent directory: '../nav_ninja/data'

In [5]:
jema_data[jema_data[security_col] == '0H6400S'] # deleted

,Security Description,Security No.,Holding August 2024,Holding July 2024,Holding June 2024,Holding May 2024,Holding April 2024,Holding March 2024,Holding February 2024,Holding January 2024,Holding December 2023,Holding November 2023,Holding October 2023,Holding September 2023,Holding August 2023,Holding July 2023,Holding June 2023,Holding May 2023,Holding April 2023,Holding March 2023,Holding February 2023,Holding January 2023,Holding December 2022,Holding November 2022,Holding October 2022,Holding September 2022,Holding August 2022,Holding July 2022,Holding June 2022,Holding May 2022,Holding April 2022,Holding March 2022


In [6]:
jan23 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='January 2023',  skiprows=9, usecols=range(5)).dropna()
dec22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='December 2022', skiprows=9, usecols=range(5)).dropna()
mar22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='March 2022',    skiprows=9, usecols=range(5)).dropna()
apr22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='April 2022',    skiprows=9, usecols=range(5)).dropna()
jun24 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='June 2024',     skiprows=9, usecols=range(5)).dropna()
apr22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='April 2022',    skiprows=9, usecols=range(5)).dropna()
# joined = pd.merge(jan23, dec22, on=security_col, how='left')
# joined = mar22.merge(apr22, on=security_col, how='left')
# joined = jun24.merge(apr22, on=security_col, how='left')

sec = '2H6442S'

display(jan23[jan23[security_col] == sec])
display(dec22[dec22[security_col] == sec])
joined = pd.merge(jan23, dec22, on=security_col, how='left')
display(joined)

,Holding,Security Description,Market Value,% of Fund,Security No.
2,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,216328.14,0.011464,2H6442S


,Holding,Security Description,Market Value,% of Fund,Security No.
2,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,212831.74,0.01128,2H6442S


,Holding_x,Security Description_x,Market Value_x,% of Fund_x,Security No.,Holding_y,Security Description_y,Market Value_y,% of Fund_y
0,17002248.27,JPM GBP LIQUIDITY LVNAV X (DIST.),17002248.27,0.901035,5819115,17002248.27,JPM GBP LIQUIDITY LVNAV X (DIST.),17002248.27,0.901101
1,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,216328.14,0.011464,2H6442S,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,212831.74,0.011280
2,262047,LUKOIL PJSC COMMON STOCK RUB,132735.25,0.007034,0H5797S,262047,LUKOIL PJSC COMMON STOCK RUB,135847.01,0.007200
3,8860909,GAZPROM PJSC COMMON STOCK RUB,207280.93,0.010985,0H5796S,8860909,GAZPROM PJSC COMMON STOCK RUB,212140.29,0.011243
4,5365084,GAZPROM PJSC ADR,25337.18,0.001343,0H6364S,5365084,GAZPROM PJSC ADR,25931.17,0.001374
5,14455036,SBERBANK OF RUSSIA PJSC PREFERENCE RUB 3,186533.96,0.009885,0H5856S,14455036,SBERBANK OF RUSSIA PJSC PREFERENCE RUB 3,190906.94,0.010118
6,2355925,SBERBANK OF RUSSIA PJSC,30182.14,0.001600,0H5855S,2355925,SBERBANK OF RUSSIA PJSC,30889.71,0.001637
7,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,186325.77,0.009874,2H6464S,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,183314.28,0.009715
8,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,186325.77,0.009874,2H6464S,103572,NOVATEK PJSC COMMON STOCK RUB 0.1,13221.25,0.000701
9,103572,NOVATEK PJSC COMMON STOCK RUB 0.1,12918.40,0.000685,2H6464S,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,183314.28,0.009715
